In [9]:
# LINHAS: Bronze -> Silver (Parquet, sem Delta)

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, explode, current_timestamp, to_date, lit
from pyspark.sql import functions as F


# --- Paths ---
today = datetime.now().strftime("%Y/%m/%d")
BRONZE_PATH = f"s3a://bronze/paradas/{today}/"   # ajuste se seu prefixo for outro (ex.: linhas_ref)
SILVER_PATH = "s3a://silver/linha_paradas/"

print(f"Lendo Bronze de: {BRONZE_PATH}")
print(f"Gravando Silver em: {SILVER_PATH}")

Lendo Bronze de: s3a://bronze/paradas/2025/11/08/
Gravando Silver em: s3a://silver/linha_paradas/


In [ ]:
# --- SparkSession (S3A/MinIO) ---
spark = (
    SparkSession.builder.appName("BronzeToSilver_Paradas_Parquet")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", True)
    .getOrCreate()
)

In [11]:
# --- Schema do JSON de paradas (map na raiz) ---
stop_struct = StructType([
    StructField("cp", LongType(),   True),   # id da parada
    StructField("np", StringType(), True),   # nome
    StructField("ed", StringType(), True),   # endereço / referência
    StructField("py", DoubleType(), True),   # latitude
    StructField("px", DoubleType(), True),   # longitude
])

schema_map = MapType(StringType(), ArrayType(stop_struct), True)


In [12]:
# lê todos os JSONs do dia: linhas_parada_*.json, por exemplo
raw = (
    spark.sparkContext
         .wholeTextFiles(f"{BRONZE_PATH.rstrip('/')}" + "/*.json")
         .toDF(["path", "raw"])
)

# parseia o JSON raiz como um MAP<cl, array<stop>>
df_parsed = raw.select(
    "path",
    F.from_json(F.col("raw"), schema_map).alias("stops_map")
)
df_parsed.printSchema()

# explode o map: uma linha por (cl, array_de_paradas)
df_line_array = df_parsed.select(
    "path",
    explode(col("stops_map")).alias("cl_str", "stops_array")
)
# cl_str é string, stops_array é array<stop_struct>

# explode o array de paradas: uma linha por parada
df_stops = df_line_array.select(
    "path",
    col("cl_str").cast("int").alias("line_id"),
    explode(col("stops_array")).alias("stop")
)
# explode em array vazio simplesmente não gera nenhuma linha, então os "[]"
# serão ignorados naturalmente.

df_clean = (
    df_stops
    .select(
        col("line_id"),
        col("stop.cp").alias("stop_id"),
        col("stop.np").alias("stop_name"),
        col("stop.ed").alias("endereco"),
        col("stop.py").alias("latitude"),
        col("stop.px").alias("longitude"),
    )
    .dropDuplicates(["line_id", "stop_id"])   # evita duplicar mesma parada/linha
    .withColumn("dt", to_date(current_timestamp()))   # snapshot diário
    .withColumn("ingest_ts", current_timestamp())
)


root
 |-- path: string (nullable = true)
 |-- stops_map: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- cp: long (nullable = true)
 |    |    |    |-- np: string (nullable = true)
 |    |    |    |-- ed: string (nullable = true)
 |    |    |    |-- py: double (nullable = true)
 |    |    |    |-- px: double (nullable = true)



In [13]:
# --- Escrita em Parquet (snapshot diário) ---
# Para snapshot do dia, normalmente usamos overwrite NA PARTIÇÃO do dia.
# Se preferir append puro, troque para .mode("append").
(
    df_clean
    .write
    .mode("overwrite")
    .partitionBy("dt", "line_id")
    .parquet(SILVER_PATH)
)

print("✅ Paradas por linha: transformação concluída e salva em Parquet na camada Silver.")


✅ Paradas por linha: transformação concluída e salva em Parquet na camada Silver.


In [14]:
df_result = spark.read.parquet(SILVER_PATH)

df_result.orderBy("dt", "line_id", "stop_id").show(20, truncate=False)


+---------+--------------------------------+--------------------------------------------------------------+----------+----------+--------------------------+----------+-------+
|stop_id  |stop_name                       |endereco                                                      |latitude  |longitude |ingest_ts                 |dt        |line_id|
+---------+--------------------------------+--------------------------------------------------------------+----------+----------+--------------------------+----------+-------+
|190011831|PARADA NICOLINO BARRA B/C       |R BARAO NICOLINO BARRA/ R JUAN GIL                            |-23.651334|-46.757487|2025-11-08 20:10:54.485713|2025-11-08|1      |
|450011801|PARADA DAS BELEZAS B/C          |R ARISTODEMO GAZZOTTI/ AV TOMAS DE SOUSA                      |-23.642592|-46.738745|2025-11-08 20:10:54.485713|2025-11-08|1      |
|450011846|PARADA AABB B/C                 |R ADELINO DE MAGALHAES/ R EMILIA MORGADO MUNHOZ GARCIA        |-23.649323|-4